This notebook was originally developed by Conor K. Corbin, modified by Minh Nguyen

### Description:
- Query the validation cohort with admitted ED patients
- similar to `1.1_cohort`

**Output**: validation `6_1_cohort1`

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 
from datetime import datetime

In [2]:
from google.cloud import bigquery
from google.cloud.bigquery import dbapi

### THIS IS MEANT TO RUN ON NERO GCP Jupyter notebook
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/jupyter/.config/gcloud/application_default_credentials.json'

# FOR LOCAL COMPUTER:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\User\AppData\Roaming\gcloud\application_default_credentials.json' 

os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101' 
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


/opt/conda/lib/python3.7/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
datadir6 = "../../DataTD/validation"
valdir = "../../OutputTD/6_validation"

In [4]:
pd.options.display.max_rows = 100

### Query ADT and return rows where patient encounter id is associated with an ED visit
* Get ER stays that BEGIN between 2015 and 2020. Some of our index times will be in 2021 because the ER visit will start on 12-31-2019 and ADT has 2021 data.
* For validation cohort, get between 2020 and 2021. Will filter to get only April 2020 to 2021 (or only take csn that are not already in previously used cohort).
* Assumes patient encounters associated with an ED visit START with an ED visit, which is probably always true-- But might want to check this. 
* Assumes we can trust pat_enc_csn_id_coded as a patient encounter - which we can't always.  Encounters are weird, and sometimes multiple pat_enc_csn_id_coded's will overlap in time for a particular patient (Stephen knows more about this). UPDATE - ER encounters should have unique csns. This was checked toward the end

In [5]:
query = """

WITH er_admit_times AS (
SELECT pat_enc_csn_id_coded, MIN(EXTRACT(YEAR FROM event_time_jittered_utc)) admit_year
FROM shc_core_2021.adt
WHERE pat_class_c = "112"
AND pat_service = "Emergency"
GROUP BY pat_enc_csn_id_coded
)

SELECT er.anon_id, er.pat_enc_csn_id_coded, er.effective_time_jittered_utc,
        er.base_pat_class_c, er.pat_lv_of_care, er.pat_class, er.event_type
           
FROM shc_core_2021.adt as er
INNER JOIN er_admit_times
USING (pat_enc_csn_id_coded)
WHERE er_admit_times.admit_year BETWEEN 2020 AND 2022
ORDER BY anon_id, pat_enc_csn_id_coded, effective_time_jittered_utc 
"""
query_job =client.query(query)
df=query_job.to_dataframe()

# order by: changed from event_time to effective_time

In [6]:
print(df.shape)
df.head()

(1084119, 7)


,anon_id,pat_enc_csn_id_coded,effective_time_jittered_utc,base_pat_class_c,pat_lv_of_care,pat_class,event_type
0,JC1000116,131283853143,2020-03-01 12:40:00+00:00,3.0,None,Emergency Services,Admission
1,JC1000116,131283853143,2020-03-01 12:51:00+00:00,NaN,None,Emergency Services,Transfer In
2,JC1000116,131283853143,2020-03-01 12:51:00+00:00,NaN,None,Emergency Services,Transfer Out
3,JC1000116,131283853143,2020-03-01 12:55:00+00:00,NaN,None,Emergency Services,Patient Update
4,JC1000116,131283853143,2020-03-01 17:07:00+00:00,NaN,None,Emergency Services,Discharge


In [7]:
# change columns to datetime, if read from locally stored data (if queried directly from BQ, already in datetime format)
df.effective_time_jittered_utc = pd.to_datetime(df.effective_time_jittered_utc)

### Get counts on unique patient trajectories 
where patients were sent throughout their stay - this is at the granularity of inpatient vs observation vs discharged straight from ER - not the unit they are sent to.

Trajectories of patient encounter ids.
* Most get discharged directly from ER.
* Next most common is ER -> inpatient (which is what we're focusing on). 
* Next is ER to observation
* Next is ER to observation to inpatient etc. 

In [8]:
# Filter for rows where pat_class changes = meaning they don't get discharged directly from the ER
df_change = df[~df['base_pat_class_c'].isna()]
print(len(df)) #
print(len(df_change)) #

1084119
208134


In [9]:
# Collapse pat class on patient encounter id, create trajectories, group by trajectories, count patient encounters with those trajectories. 
pd.set_option("display.max_colwidth", None)

df_change[['pat_enc_csn_id_coded', 'pat_class']].groupby('pat_enc_csn_id_coded').agg(
{'pat_class' : lambda x: ' -> '.join([c for c in x])}).reset_index().groupby('pat_class').agg(
{'pat_enc_csn_id_coded' : 'count'}).reset_index().sort_values('pat_enc_csn_id_coded', ascending=False)

,pat_class,pat_enc_csn_id_coded
0,Emergency Services,120258
3,Emergency Services -> Inpatient,22744
12,Emergency Services -> Observation,10654
13,Emergency Services -> Observation -> Inpatient,4831
1,Emergency Services -> Bedded Outpatient (corrections only),1450
6,Emergency Services -> Inpatient -> Observation,449
10,Emergency Services -> OP Surgery/Procedure,373
2,Emergency Services -> Bedded Outpatient (corrections only) -> Inpatient,134
11,Emergency Services -> OP Surgery/Procedure -> Inpatient,104
4,Emergency Services -> Inpatient -> Bedded Outpatient (corrections only),102


### Patient Level of Care Column is the indicator we want - 
It seems to be missing a lot 
Update: LOC not always on the same row as the change in pat code status ie inpatient vs observation vs emergency services

Filter for patients with an inpatient code immediately after emergency services and create trajectories

In [10]:
import datetime

def has_inpatient_code(arr):
    for a in arr:
        if a == 'Inpatient':
            return True
    return False

def has_inpatient_code_after_er(arr):
    """Assumes arr is ordered by time"""
    has_er = False
    for a in arr:
        if a == 'Emergency Services':
            has_er = True
        elif a == 'Inpatient' and has_er == True:
            return True
        else:
            has_er = False
        
    return False

def get_trajectory(arr):
    # Creates trajectory but only adds to path when level of care changes
    traj = []
    for i, a in enumerate(arr):
        if len(traj) == 0:
            traj.append(a)
        elif a != traj[-1]:
            traj.append(a)
    return ' -> '.join(traj)

# Get a set of patient_encounter_ids that have an inpatient code
df_temp = df_change[['pat_enc_csn_id_coded', 'pat_class']].groupby('pat_enc_csn_id_coded').agg(
{'pat_class' : has_inpatient_code_after_er}).reset_index()
inpatient_ids = set(df_temp[df_temp['pat_class'] == True]['pat_enc_csn_id_coded'].values)

# Filter original df for patients in this set and create level of care trajectories.
df_lofc = df[df['pat_enc_csn_id_coded'].isin(inpatient_ids)]

# Get df of csn_ids and admit timestamps
df_admit_times = df_change[df_change['pat_enc_csn_id_coded'].isin(inpatient_ids)]
df_admit_times = df_admit_times[df_admit_times['pat_class'] == 'Inpatient'].groupby(
    'pat_enc_csn_id_coded').first().reset_index()[['pat_enc_csn_id_coded', 'effective_time_jittered_utc']].rename(
    columns={'effective_time_jittered_utc' : 'admit_time_jittered'})
df_admit_times.head()

# Should now be querying adt again for anon_id that match each er CSN id and then look ahead 24 hours to mitigate overlapping csn issue

# Merge to df_lofc and create column called time_since_admit
df_lofc = pd.merge(df_lofc, df_admit_times, how='left', on='pat_enc_csn_id_coded')
df_lofc['time_since_admit'] = df_lofc.apply(lambda x: x.effective_time_jittered_utc - x.admit_time_jittered, axis=1)

# Filter df_lofc so that we only look 24 hours into admission
df_lofc = df_lofc[df_lofc['time_since_admit'] < datetime.timedelta(hours=24)]

print(len(df_lofc)) # 

218783


### Print the trajectories 24 hours into admission

In [11]:
# Merge this to df_lofc and 
df_traj = df_lofc[['pat_enc_csn_id_coded', 'pat_lv_of_care']].dropna().groupby(
'pat_enc_csn_id_coded').agg({'pat_lv_of_care' : get_trajectory}).reset_index().groupby(
'pat_lv_of_care').count().reset_index().sort_values('pat_enc_csn_id_coded', ascending=False)

# Print cause these trajectories are long
for i in range(len(df_traj)):
    print(df_traj['pat_lv_of_care'].values[i], ' : ', df_traj['pat_enc_csn_id_coded'].values[i])

# Count number of encounters with a trajectory. 
print('Number of encounters with a trajectory : ', df_traj['pat_enc_csn_id_coded'].sum())
print('Total Number of inpatient encounters : ', len(inpatient_ids))

Acute Care (Assessment or intervention q4-8)  :  17933
Critical Care  :  1834
IICU/Intermediate Care (Assessment or intervention q2-4)  :  964
Critical Care -> Acute Care (Assessment or intervention q4-8)  :  621
Acute Care (Assessment or intervention q4-8) -> IICU/Intermediate Care (Assessment or intervention q2-4)  :  530
IICU/Intermediate Care (Assessment or intervention q2-4) -> Acute Care (Assessment or intervention q4-8)  :  391
Acute Care (Assessment or intervention q4-8) -> Critical Care  :  366
Critical Care -> IICU/Intermediate Care (Assessment or intervention q2-4)  :  145
Acute Care (Assessment or intervention q4-8) -> IICU/Intermediate Care (Assessment or intervention q2-4) -> Acute Care (Assessment or intervention q4-8)  :  113
IICU/Intermediate Care (Assessment or intervention q2-4) -> Critical Care  :  101
Acute Care (Assessment or intervention q4-8) -> Critical Care -> Acute Care (Assessment or intervention q4-8)  :  69
Acute Care (Assessment or intervention q4-8) -> I

### Create Labelling function for highest level of care with 24 hours of admit
For each csn id, we'll create positive or negative labels based on whether within 24 hours of admit they have a critical care label in pat_lv_of_care.  This means that if they are originally place in critical care but then sent to acute care we'll still label them as crit care.  Label is thus the max level of care within 24 ours of admit

Output dataframe should have anon_id, csn_id, admit_time, label

NOTE : this isn't completley correct because i've grouped on pat_enc_csn_id_coded.  I really should be taking the anon_id from each er csn id and looking ahead 24 hours in the adt table to see if there exist other csn id's associated with the encounter where the level of care changes... 

UPDATE: OK to group by pat_enc_csn_id_coded, take the first admit time for each visit, first anon_id only matter if there are multiple anon_ids with a CSN. This is not the case in our data, checked.

In [12]:
def was_placed_in_critical_care(arr):
    """Returns true if patient placed in crtical care within 24 hours of admit
       Assumes we have already done the 24 hours logic
       Assumes no overlapping csn ids... """
    for a in arr:
        if a == 'Critical Care':
            return 1
    return 0

In [13]:
# group by anon_id and csn
df_labels = df_lofc.groupby(['anon_id','pat_enc_csn_id_coded']).agg({ #cohort0_all_anon_ids
    'admit_time_jittered' : 'first',
    'pat_lv_of_care' : was_placed_in_critical_care}).rename(
    columns={"pat_lv_of_care" : 'label'}).reset_index()[['anon_id', 'pat_enc_csn_id_coded', 'admit_time_jittered', 'label']]

print(len(df_labels)) # 55168

# Save to .csv and read back as necessary
df_labels.to_csv(os.path.join(valdir,"6_1_cohort1.csv"), index=False)

# check labels
df_labels.groupby('label').count() # 23357

23361


,anon_id,pat_enc_csn_id_coded,admit_time_jittered
label,,,
0,19991,19991,19991
1,3370,3370,3370


In [14]:
# make sure you only see 1's in the anon_id column!! (or else there are multiple anon_ids for a csn)
df_labels[['anon_id','pat_enc_csn_id_coded']].groupby('pat_enc_csn_id_coded').nunique().reset_index().sort_values('anon_id')

,pat_enc_csn_id_coded,anon_id
0,131250027848,1
15578,131307582273,1
15577,131307581649,1
15576,131307580529,1
15575,131307576616,1
...,...,...
7782,131293552052,1
7781,131293551859,1
7780,131293551762,1
7778,131293550890,1


### Save to big query, the orginial 1_1_cohort = cohort _init_

In [15]:
# read back as necessary
df_labels = pd.read_csv(os.path.join(valdir, '6_1_cohort1.csv'), index_col=False)
print(len(df_labels))
print(list(df_labels.columns))

23361
['anon_id', 'pat_enc_csn_id_coded', 'admit_time_jittered', 'label']


In [16]:
table_schema = [{'name' : 'anon_id', 'type' : 'STRING'},
                {'name' : 'pat_enc_csn_id_coded', 'type' : 'INTEGER'},
                {'name' : 'admit_time_jittered', 'type' : 'TIMESTAMP'},
                {'name' : 'label', 'type' : 'INTEGER'}]
                       
DATASET_NAME = 'triageTD'
TABLE_NAME = '6_1_cohort1'
df_labels.to_gbq(destination_table='triageTD.%s' % TABLE_NAME,
                 project_id='som-nero-phi-jonc101',
                 table_schema=table_schema,
                 if_exists='replace')

/opt/conda/lib/python3.7/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
1it [00:02,  2.96s/it]
